In [ ]:
import numpy as np
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from matplotlib import cm
import torch
from torch.optim.lr_scheduler import StepLR
import albumentations as A
from pathSeg.ml.hovernet import HoVerNet, loss_hovernet, post_process_batch_hovernet
from pathSeg.ml.utils import dice_score
from pathSeg.utils import plot_segmentation
import pytorch_model_summary as tms
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
import scipy.io
from glob import glob
from PIL import Image   
from loguru import logger
from torchvision.transforms import ToTensor
import cv2
import random
from sklearn.model_selection import train_test_split
from shapely.geometry import Polygon, MultiPolygon
import cv2
import xml.etree.ElementTree as ET
import os
import torchvision
print(f"GPUs used:\t{torch.cuda.device_count()}")
device = torch.device("cuda:6")
print(f"Device:\t\t{device}")

In [ ]:
class_list=['유형1','유형2']
params={'image_size':1024,
        'lr':1e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':1,
        'epochs':1000,
        'n_classes':None,
        'data_path':'../../data/segmentation/BR/BRNT/',}

tf = ToTensor()

In [ ]:

def binary_mask_to_polygon(binary_mask):
    # binary_mask는 2차원 numpy array여야 합니다.
    # Contours를 찾습니다.
    contours=[]
    k11=cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    for i in range(1,binary_mask.max()):
        mask=np.where(binary_mask==i,1,0).astype(np.uint8)
        mask=cv2.morphologyEx(mask, cv2.MORPH_OPEN,k11 )
        try:
            contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        except:
            contour=()
        if(len(contour)!=0):
            contours.append(contour[0])
    polygons = []
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) >= 10]
    for contour in contours:
        # 각 contour를 polygon으로 변환
        if len(contour) >= 3:  # 유효한 polygon을 만들기 위해서 최소한 3개의 점이 필요합니다.
            poly = Polygon(shell=[(point[0][0], point[0][1]) for point in contour])
            polygons.append(poly)
    
    if len(polygons) > 1:
        # 여러 개의 polygon이 있을 경우 MultiPolygon으로 변환
        return MultiPolygon(polygons)
    elif len(polygons) == 1:
        return MultiPolygon(polygons)
    else:
        return None
    
def mask2polygon(mask):
    poly=binary_mask_to_polygon(mask)

    polygon_arrays = []

    if poly!=None:
        for polygon in poly.geoms:
            exterior_coords = np.array(polygon.exterior.coords)
            polygon_arrays.append(exterior_coords)
  
    return  polygon_arrays

def polygon2asap(label_polygon,class_list,save_path):
    # 루트 엘리먼트 생성
    root = ET.Element("ASAP_Annotations")
    # Annotations 엘리먼트 생성 및 루트에 추가
    annotations = ET.SubElement(root, "Annotations")
    for i in range(len(label_polygon)):
        
        for j in range(len(label_polygon[i])):
            annotation = ET.SubElement(annotations, "Annotation", Name=class_list[i], Type="Polygon", PartOfGroup="None", Color="#F4FA58")
            coordinates = ET.SubElement(annotation, "Coordinates")
            for k in range(len(label_polygon[i][j])):
                ET.SubElement(coordinates, "Coordinate", Order=str(k), X=str(float(label_polygon[i][j][k,0])), Y=str(float(label_polygon[i][j][k,1])))
            
            
    tree = ET.ElementTree(root)
    tree.write(save_path)
    
def polygon2mask(image_shape, polygons):
    # 빈 마스크 생성 (모든 채널을 0으로 초기화)
    mask=np.zeros((image_shape[0], image_shape[1]), dtype=np.uint8)
    
    for polygon in polygons:
        polygon = np.array(polygon, dtype=np.int32)
        cv2.fillPoly(mask, [polygon], 255)

    return mask

In [ ]:

xml_path='../../result/nucleus_segmentation/BR/BRNT/'
image_list=[]
category_list=[]
for k in range(len(class_list)):
    img_list=glob(params['data_path']+class_list[k]+'/*.jpeg')
    for i in range(len(img_list)):
        image_list.append(img_list[i])
        category_list.append(class_list[k])

    
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list
        self.tf= ToTensor()
    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        path=self.img_path[idx]
        image=self.tf(Image.open(self.img_path[idx]))
        label=self.label[idx]
        return image,label,path

dataset = CustomDataset(image_list, category_list)

dataloader = DataLoader(dataset, batch_size=params['batch_size'], shuffle=False)

In [ ]:
# load the model
hovernet = HoVerNet(n_classes=params['n_classes']).to(device)

opt = torch.optim.Adam(hovernet.parameters(), lr = params['lr'], betas=(params['beta1'], params['beta2']))
# learning rate scheduler to reduce LR by factor of 10 each 25 epochs
scheduler = StepLR(opt, step_size=25, gamma=0.1)
checkpoint = torch.load("../../model/pathSeg/hovernet_binary_qupath_best_perf.pt", map_location=device)
hovernet.load_state_dict(checkpoint)

In [ ]:
hovernet.eval()

ims = None
mask_truth = None
mask_pred = None
tissue_types = []
path_list=[]
with torch.no_grad():
    for i, data in enumerate(tqdm(dataloader)):
        # send the data to the GPU
        images = data[0].float().to(device)
        path=data[2]
        label=data[1]
        # pass thru network to get predictions
        outputs = hovernet(images)
        preds_detection = post_process_batch_hovernet(outputs, n_classes=params['n_classes'])
        polygon=mask2polygon(preds_detection[0])
        label_polygon=[polygon]
        save_path=xml_path+label[0]+'/'+os.path.basename(path[0]).split('.')[0]+'.xml'
        polygon2asap(label_polygon,['Cell_nucleus'],save_path)
        topilimage = torchvision.transforms.ToPILImage()
        np_image=np.transpose(np.array(images[0].cpu().detach()),(1,2,0))*255
        overlay=np_image*0.5
        overlay[...,1]=overlay[...,1]+polygon2mask((1024,1024),polygon)*0.5
        overlay=overlay.astype(np.uint8)
        Image.fromarray(overlay).save('../../result/nucleus_segmentation/BR/BRNT/overlay/'+os.path.basename(path[0]).split('.')[0]+'.png')

In [ ]:
hovernet.eval()

ims = None
mask_truth = None
mask_pred = None
tissue_types = []
path_list=[]
with torch.no_grad():
    for i, data in enumerate(tqdm(dataloader)):
        # send the data to the GPU
        images = data[0].float().to(device)
        path=data[2]
        label=data[1]
        # pass thru network to get predictions
        outputs = hovernet(images)
        preds_detection = post_process_batch_hovernet(outputs, n_classes=params['n_classes'])
        polygon=mask2polygon(preds_detection[0])

        topilimage = torchvision.transforms.ToPILImage()
        np_image=np.transpose(np.array(images[0].cpu().detach()),(1,2,0))*255
        overlay=np_image
        overlay[...,1]=overlay[...,1]+polygon2mask((1024,1024),polygon)*0.5
        overlay=overlay.astype(np.uint8)
        Image.fromarray(overlay)

In [ ]:
hovernet.eval()

ims = None
mask_truth = None
mask_pred = None
tissue_types = []
name_list=[]
with torch.no_grad():
    for i, data in tqdm(enumerate(dataloader)):
        # send the data to the GPU
        images = data[0].float().to(device)
        file_name=os.path.basename(data[1][0])
        name_list.append(file_name)
        # pass thru network to get predictions
        outputs = hovernet(images)
        preds_detection = post_process_batch_hovernet(outputs, n_classes=params['n_classes'])
        
        if i == 0:
            ims = data[0].numpy()
            mask_pred = preds_detection

        else:
            ims = np.concatenate([ims, data[0].numpy()], axis=0)
            mask_pred = np.concatenate([mask_pred, preds_detection], axis=0)

In [ ]:
ims1 = np.moveaxis(ims, 1, 3)
n = 5
mask_pred1=mask_pred[:,np.newaxis,:,:] 
ix = np.random.choice(np.arange(20), size = n)
fig, ax = plt.subplots(nrows = n, ncols = 2, figsize = (22, 10*n))
for i, index in enumerate(ix):
    ax[i, 0].imshow(ims1[index, ...])
    ax[i, 1].imshow(ims1[index, ...])
    plot_segmentation(ax = ax[i, 1], masks = mask_pred1[index, ...])
        
for a in ax.ravel(): 
    a.get_xaxis().set_ticks([])
    a.get_yaxis().set_ticks([])
ax[0, 0].set_title("image")
ax[0, 1].set_title("Prediction")
plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.colors import TABLEAU_COLORS
TABLEAU_COLORS.values()